In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from glob import glob
from tqdm import tqdm

os.environ['CUDA_VISIBLE_DEVICES'] = "1"

train_val_df = pd.read_csv("../../data/train_val.csv")
test_df = pd.read_csv("../../data/test.csv")

# Extract the paths and labels
label_columns = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'No Finding']
train_val_labels = train_val_df[label_columns].values
test_labels = test_df[label_columns].values

In [2]:
train_val_df['Paths'] = train_val_df['Paths'].apply(lambda x : x.split("../../../")[1])
test_df['Paths'] = test_df['Paths'].apply(lambda x : x.split("../../../")[1])

In [3]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, labels, transform=None):
        self.dataframe = dataframe
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.dataframe.iloc[idx]['Paths'])
        image = Image.open(img_name).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_val_dataset = CustomDataset(train_val_df, train_val_labels, transform=transform)
test_dataset = CustomDataset(test_df, test_labels, transform=transform)

train_loader = DataLoader(train_val_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)


In [4]:
import torch.nn as nn
import torchvision.models as models

class ResNetModified(nn.Module):
    def __init__(self):
        super(ResNetModified, self).__init__()
        original_model = models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 15),
            nn.Sigmoid()  # Sigmoid for multi-label classification
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetModified()

model = nn.DataParallel(model)
model = model.to(device)


/userHome/userhome4/seungyeon/miniconda3/envs/nih_syn/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/userHome/userhome4/seungyeon/miniconda3/envs/nih_syn/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm

criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Train_loader Epoch: {epoch}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Evaluate the model
    model.eval()
    all_labels = []
    all_outputs = []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f"Test_loader Epoch: {epoch}"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())

    all_labels = np.vstack(all_labels)
    all_outputs = np.vstack(all_outputs)
    all_outputs = (all_outputs > 0.5).astype(int)

    accuracy = accuracy_score(all_labels, all_outputs)
    roc_auc = roc_auc_score(all_labels, all_outputs)
    f1 = f1_score(all_labels, all_outputs, average='macro')

    print(f"Accuracy: {accuracy}, F1-Score: {f1}, Roc_Auc: {roc_auc}")


root_save_path = "save"
if not os.path.exists(root_save_path):
    os.makedirs(root_save_path)
    
model_save_path =os.path.join(root_save_path, "resnet_modified.pth")

# Save the model
torch.save(model.state_dict(), model_save_path)


Train_loader Epoch: 0:   0%|          | 0/338 [00:14<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 23.68 GiB total capacity; 6.17 GiB already allocated; 361.94 MiB free; 6.39 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
def evaluate_and_save_results(model, test_loader, threshold=0.5):
    model.eval()
    all_labels = []
    all_outputs = []
    all_filenames = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())
            all_filenames.extend(test_loader.dataset.dataframe.iloc[:len(images)]['Paths'].tolist())

    all_labels = np.vstack(all_labels)
    all_outputs = np.vstack(all_outputs)
    binary_outputs = (all_outputs > threshold).astype(int)

    results_df = pd.DataFrame({
        'filename': all_filenames,
        'true_labels': list(all_labels),
        'predicted_labels': list(binary_outputs),
        'predicted_probabilities': list(all_outputs)
    })

    results_df.to_csv('evaluation_results.csv', index=False)

    accuracy = accuracy_score(all_labels, binary_outputs)
    # precision = precision_score(all_labels, binary_outputs, average='micro')
    recall = roc_auc_score(all_labels, binary_outputs)
    f1 = f1_score(all_labels, binary_outputs, average='micro')

    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

# Evaluate the model and save results
evaluate_and_save_results(model, test_loader, threshold=0.5)


UnidentifiedImageError: cannot identify image file './nih_resize_all/00002176_000.png'